# Dadbot: Dad's memorial bot based on RASA (old style)

## Starting Jupyter Notebook with necessary imports

In [1]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')


# Installations
* Rasa NLU
* Rasa Core
* SpaCy Language Model

In [2]:
import sys
python = sys.executable

# In your environment run:
#!{python} -m pip install -U rasa_core==0.14.5 rasa_nlu[spacy];
#!{python} -m pip install spacy==2.2.1

In [3]:
#!{python} -m spacy download es_core_news_md

## Downloading the Spanish Language Model

In [4]:
!{python} -m spacy link es_core_news_md es --force;

✔ Linking successful
/home/debian/.conda/envs/rasa/lib/python3.7/site-packages/es_core_news_md -->
/home/debian/.conda/envs/rasa/lib/python3.7/site-packages/spacy/data/es
You can now load the model via spacy.load('es')


## Import the Installations

In [5]:
import rasa_nlu
import rasa_core

# 1. Teaching the bot to understand user inputs using Rasa NLU

## Training the NLU Model.

In [7]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config
import spacy

#spacy_parser = spacy.load('es_core_news_md')
#nlp = spacy.load('es')

# loading the nlu training samples
training_data = load_data("data/nlu/nlu-papaito.md")

# trainer to educate our pipeline
trainer = Trainer(config.load("config_simple.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

INFO:rasa_nlu.training_data.loading:Training data format of data/nlu/nlu-papaito.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 618 (24 distinct intents)
	- Found intents: 'ask_nombre', 'inform_estado', 'ask_vivienda', 'ask_gustos', 'greet', 'inform_pregunta', 'ask_deporte_bici', 'ask_cuenta_algo', 'ask_deporte_futbol', 'inform_comida', 'goodbye', 'ask_help', 'ask_ultimas_noticias', 'inform_ciudad', 'ask_estado', 'ask_razon', 'ask_comida', 'mood_affirm', 'mood_unhappy', 'ask_actividad', 'ask_pais', 'mood_deny', 'ask_ocupacion', 'mood_happy'
	- entity examples: 106 (9 distinct entities)
	- found entities: 'actividad', 'salud', 'cosa', 'comida', 'ciudad', 'horario', 'deporte', 'famoso', 'plato'

INFO:rasa_nlu.model:Starting to train component WhitespaceTokenizer
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component RegexFeaturizer
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:S


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:rasa_nlu.classifiers.embedding_intent_classifier:Accuracy is updated every 10 epochs
Epochs: 100%|██████████| 300/300 [00:11<00:00, 25.22it/s, loss=0.041, acc=0.985]
INFO:rasa_nlu.classifiers.embedding_intent_classifier:Finished training embedding classifier, loss=0.041, train accuracy=0.985
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Successfully saved model into '/home/debian/workspace/Dadbot/models/nlu/default/current'


## Evaluating the NLU model on a random text

In [8]:
# A helper function for prettier output

def pprint(o):   
    print(json.dumps(o, indent=2))
    
pprint(interpreter.parse("dejándome el coche"))

{
  "intent": {
    "name": "ask_help",
    "confidence": 0.9321062564849854
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "ask_help",
      "confidence": 0.9321062564849854
    },
    {
      "name": "ask_cuenta_algo",
      "confidence": 0.1614149808883667
    },
    {
      "name": "inform_pregunta",
      "confidence": 0.12934404611587524
    },
    {
      "name": "ask_nombre",
      "confidence": 0.12723326683044434
    },
    {
      "name": "ask_ocupacion",
      "confidence": 0.09158955514431
    },
    {
      "name": "inform_estado",
      "confidence": 0.06412962079048157
    },
    {
      "name": "greet",
      "confidence": 0.05872807651758194
    },
    {
      "name": "ask_razon",
      "confidence": 0.03737158700823784
    },
    {
      "name": "ask_vivienda",
      "confidence": 0.03010689467191696
    },
    {
      "name": "ask_ultimas_noticias",
      "confidence": 0.023918434977531433
    }
  ],
  "text": "dej\u00e1ndome el coche"
}


## Evaluating the NLU model on a test data
(Here we are using the data at hand i.e nlu.md but it isr recommended to use unseen data)

In [10]:
from rasa_nlu.test import run_evaluation

run_evaluation("data/nlu/nlu-papaito.md", model_directory)

INFO:tensorflow:Restoring parameters from /home/debian/workspace/Dadbot/./models/nlu/default/current/component_5_EmbeddingIntentClassifier.ckpt
INFO:rasa_nlu.training_data.loading:Training data format of data/nlu/nlu-papaito.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 618 (24 distinct intents)
	- Found intents: 'ask_nombre', 'inform_estado', 'ask_vivienda', 'ask_gustos', 'greet', 'inform_pregunta', 'ask_deporte_bici', 'ask_cuenta_algo', 'ask_deporte_futbol', 'inform_comida', 'goodbye', 'ask_help', 'ask_ultimas_noticias', 'inform_ciudad', 'ask_estado', 'ask_razon', 'ask_comida', 'mood_affirm', 'mood_unhappy', 'ask_actividad', 'ask_pais', 'mood_deny', 'ask_ocupacion', 'mood_happy'
	- entity examples: 106 (9 distinct entities)
	- found entities: 'actividad', 'salud', 'cosa', 'comida', 'ciudad', 'horario', 'deporte', 'famoso', 'plato'

INFO:rasa_nlu.test:Running model for predictions:
100%|██████████| 618/618 [00:00<00:00, 961.85it/s]
INFO:r

{'intent_evaluation': {'predictions': [{'text': '¡ey!',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9128955602645874},
   {'text': '¡eyyy!',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9101116061210632},
   {'text': 'hola tú',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9442994594573975},
   {'text': 'hola',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9406754970550537},
   {'text': 'hola de nuevo',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9341384768486023},
   {'text': 'buenos días',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.952172040939331},
   {'text': 'buenos días por la mañana',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9523468613624573},
   {'text': 'buenas tardes',
    'intent': 'greet',
    'predicted': 'goodbye',
    'confidence': 0.6896316409111023},
   {'text': 'buenas tardes por las tardes',
    

# 2. Teaching the bot to respond using Rasa Core

##  Visualising the Training Data

In [ ]:

#!apt-get -qq install -y graphviz libgraphviz-dev pkg-config;
#!breq install graphviz

#!conda install -y -n rasa pygraphviz pkg-config;

In [11]:
from IPython.display import Image
from rasa_core.agent import Agent
 
agent = Agent('domain-papaito.yml')
agent.visualize("data/core/stories-papaito.md", "story_graph.png", max_history=2)
Image(filename="story_graph.png")

INFO:apscheduler.scheduler:Scheduler started
Processed Story Blocks: 100%|██████████| 28/28 [00:00<00:00, 1279.13it/s, # trackers=1]


## Training a Dialogue Model

In [12]:
from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy, FormPolicy
from rasa_core.agent import Agent

# this will catch predictions the model isn't very certain about
# there is a threshold for the NLU predictions as well as the action predictions
fallback = FallbackPolicy(fallback_action_name="utter_unclear",
                          core_threshold=0.2,
                          nlu_threshold=0.1)

agent = Agent('domain-papaito.yml',
              policies=[MemoizationPolicy(max_history=5),
                        KerasPolicy(validation_split=0.1,epochs=630),
                        FormPolicy(),
                        fallback])

# loading our neatly defined training dialogues
training_data = agent.load_data('data/core/stories-papaito.md')

agent.train(training_data)

agent.persist('models/dialogue')

Processed Story Blocks: 100%|██████████| 28/28 [00:00<00:00, 180.45it/s, # trackers=20]
Processed actions: 76it [00:00, 2373.44it/s, # examples=72]
INFO:rasa_core.policies.keras_policy:Fitting model with 629 total samples and a validation split of 0.1


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 82)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                14720     
_________________________________________________________________
dense (Dense)                (None, 36)                1188      
_________________________________________________________________
activation (Activation)      (None, 36)                0         
Total params: 15,908
Trainable params: 15,908
Non-trainable params: 0
_________________________________________________________________
Epoch 1/630
629/629 [==============================] - 1s 2ms/sample - loss: 3.5149 - acc: 0.1638
Epoch 2/630
629/629 [==============================] - 0s 120us/sample - loss: 3.2713 - acc: 0.4610
Epoch 3/630
629/629 [==============================] - 0s 130us/sample -

629/629 [==============================] - 0s 155us/sample - loss: 0.3980 - acc: 0.9205
Epoch 74/630
629/629 [==============================] - 0s 132us/sample - loss: 0.3914 - acc: 0.9189
Epoch 75/630
629/629 [==============================] - 0s 137us/sample - loss: 0.3757 - acc: 0.9332
Epoch 76/630
629/629 [==============================] - 0s 144us/sample - loss: 0.3625 - acc: 0.9237
Epoch 77/630
629/629 [==============================] - 0s 137us/sample - loss: 0.3617 - acc: 0.9285
Epoch 78/630
629/629 [==============================] - 0s 144us/sample - loss: 0.3708 - acc: 0.9348
Epoch 79/630
629/629 [==============================] - 0s 147us/sample - loss: 0.3600 - acc: 0.9157
Epoch 80/630
629/629 [==============================] - 0s 140us/sample - loss: 0.3445 - acc: 0.9348
Epoch 81/630
629/629 [==============================] - 0s 142us/sample - loss: 0.3216 - acc: 0.9348
Epoch 82/630
629/629 [==============================] - 0s 140us/sample - loss: 0.3118 - acc: 0.9332
Epo

629/629 [==============================] - 0s 128us/sample - loss: 0.1457 - acc: 0.9618
Epoch 154/630
629/629 [==============================] - 0s 122us/sample - loss: 0.1099 - acc: 0.9746
Epoch 155/630
629/629 [==============================] - 0s 122us/sample - loss: 0.1012 - acc: 0.9825
Epoch 156/630
629/629 [==============================] - 0s 123us/sample - loss: 0.1204 - acc: 0.9714
Epoch 157/630
629/629 [==============================] - 0s 125us/sample - loss: 0.0900 - acc: 0.9857
Epoch 158/630
629/629 [==============================] - 0s 133us/sample - loss: 0.1245 - acc: 0.9666
Epoch 159/630
629/629 [==============================] - 0s 137us/sample - loss: 0.1055 - acc: 0.9777
Epoch 160/630
629/629 [==============================] - 0s 137us/sample - loss: 0.0921 - acc: 0.9777
Epoch 161/630
629/629 [==============================] - 0s 138us/sample - loss: 0.1225 - acc: 0.9634
Epoch 162/630
629/629 [==============================] - 0s 133us/sample - loss: 0.1033 - acc: 0

629/629 [==============================] - 0s 129us/sample - loss: 0.0648 - acc: 0.9762
Epoch 234/630
629/629 [==============================] - 0s 125us/sample - loss: 0.0877 - acc: 0.9714
Epoch 235/630
629/629 [==============================] - 0s 127us/sample - loss: 0.0856 - acc: 0.9777
Epoch 236/630
629/629 [==============================] - 0s 123us/sample - loss: 0.0731 - acc: 0.9809
Epoch 237/630
629/629 [==============================] - 0s 140us/sample - loss: 0.0658 - acc: 0.9777
Epoch 238/630
629/629 [==============================] - 0s 153us/sample - loss: 0.0776 - acc: 0.9762
Epoch 239/630
629/629 [==============================] - 0s 142us/sample - loss: 0.0826 - acc: 0.9809
Epoch 240/630
629/629 [==============================] - 0s 152us/sample - loss: 0.0608 - acc: 0.9809
Epoch 241/630
629/629 [==============================] - 0s 138us/sample - loss: 0.0661 - acc: 0.9777
Epoch 242/630
629/629 [==============================] - 0s 125us/sample - loss: 0.0558 - acc: 0

629/629 [==============================] - 0s 150us/sample - loss: 0.0639 - acc: 0.9746
Epoch 314/630
629/629 [==============================] - 0s 135us/sample - loss: 0.0471 - acc: 0.9841
Epoch 315/630
629/629 [==============================] - 0s 130us/sample - loss: 0.0486 - acc: 0.9809
Epoch 316/630
629/629 [==============================] - 0s 138us/sample - loss: 0.0571 - acc: 0.9825
Epoch 317/630
629/629 [==============================] - 0s 131us/sample - loss: 0.0567 - acc: 0.9841
Epoch 318/630
629/629 [==============================] - 0s 132us/sample - loss: 0.0591 - acc: 0.9793
Epoch 319/630
629/629 [==============================] - 0s 148us/sample - loss: 0.0500 - acc: 0.9841
Epoch 320/630
629/629 [==============================] - 0s 128us/sample - loss: 0.0426 - acc: 0.9873
Epoch 321/630
629/629 [==============================] - 0s 120us/sample - loss: 0.0527 - acc: 0.9841
Epoch 322/630
629/629 [==============================] - 0s 122us/sample - loss: 0.0526 - acc: 0

629/629 [==============================] - 0s 140us/sample - loss: 0.0507 - acc: 0.9809
Epoch 394/630
629/629 [==============================] - 0s 127us/sample - loss: 0.0370 - acc: 0.9857
Epoch 395/630
629/629 [==============================] - 0s 134us/sample - loss: 0.0390 - acc: 0.9873
Epoch 396/630
629/629 [==============================] - 0s 129us/sample - loss: 0.0419 - acc: 0.9857
Epoch 397/630
629/629 [==============================] - 0s 131us/sample - loss: 0.0367 - acc: 0.9873
Epoch 398/630
629/629 [==============================] - 0s 127us/sample - loss: 0.0401 - acc: 0.9825
Epoch 399/630
629/629 [==============================] - 0s 130us/sample - loss: 0.0350 - acc: 0.9889
Epoch 400/630
629/629 [==============================] - 0s 129us/sample - loss: 0.0402 - acc: 0.9841
Epoch 401/630
629/629 [==============================] - 0s 135us/sample - loss: 0.0276 - acc: 0.9889
Epoch 402/630
629/629 [==============================] - 0s 135us/sample - loss: 0.0413 - acc: 0

629/629 [==============================] - 0s 126us/sample - loss: 0.0379 - acc: 0.9841
Epoch 474/630
629/629 [==============================] - 0s 119us/sample - loss: 0.0512 - acc: 0.9809
Epoch 475/630
629/629 [==============================] - 0s 121us/sample - loss: 0.0307 - acc: 0.9921
Epoch 476/630
629/629 [==============================] - 0s 126us/sample - loss: 0.0450 - acc: 0.9841
Epoch 477/630
629/629 [==============================] - 0s 125us/sample - loss: 0.0466 - acc: 0.9762
Epoch 478/630
629/629 [==============================] - 0s 129us/sample - loss: 0.0293 - acc: 0.9905
Epoch 479/630
629/629 [==============================] - 0s 126us/sample - loss: 0.0434 - acc: 0.9793
Epoch 480/630
629/629 [==============================] - 0s 122us/sample - loss: 0.0302 - acc: 0.9873
Epoch 481/630
629/629 [==============================] - 0s 126us/sample - loss: 0.0320 - acc: 0.9889
Epoch 482/630
629/629 [==============================] - 0s 125us/sample - loss: 0.0588 - acc: 0

629/629 [==============================] - 0s 148us/sample - loss: 0.0274 - acc: 0.9873
Epoch 554/630
629/629 [==============================] - 0s 128us/sample - loss: 0.0488 - acc: 0.9793
Epoch 555/630
629/629 [==============================] - 0s 132us/sample - loss: 0.0233 - acc: 0.9905
Epoch 556/630
629/629 [==============================] - 0s 128us/sample - loss: 0.0326 - acc: 0.9889
Epoch 557/630
629/629 [==============================] - 0s 119us/sample - loss: 0.0297 - acc: 0.9889
Epoch 558/630
629/629 [==============================] - 0s 129us/sample - loss: 0.0299 - acc: 0.9889
Epoch 559/630
629/629 [==============================] - 0s 126us/sample - loss: 0.0443 - acc: 0.9825
Epoch 560/630
629/629 [==============================] - 0s 133us/sample - loss: 0.0427 - acc: 0.9841
Epoch 561/630
629/629 [==============================] - 0s 126us/sample - loss: 0.0324 - acc: 0.9841
Epoch 562/630
629/629 [==============================] - 0s 125us/sample - loss: 0.0411 - acc: 0

INFO:rasa_core.policies.keras_policy:Done fitting keras policy model
INFO:rasa_core.agent:Model directory models/dialogue exists and contains old model files. All files will be overwritten.
INFO:rasa_core.agent:Persisted model to '/home/debian/workspace/Dadbot/models/dialogue'


# Talk to your Bot

In [13]:
#Starting the Bot
from rasa_core.agent import Agent
from rasa_core.utils import EndpointConfig

action_endpoint = EndpointConfig(url="http://0.0.0.0:5055/webhook")
agent = Agent.load('models/dialogue', interpreter=model_directory, action_endpoint=action_endpoint)

#!docker run -d -p 5055:5055 --mount type=bind,source=/home/debian/workspace/Dadbot/actions/actions.py,target=/app/actions rasa/rasa-sdk

INFO:tensorflow:Restoring parameters from /home/debian/workspace/Dadbot/./models/nlu/default/current/component_5_EmbeddingIntentClassifier.ckpt


## Creating the voice synthesizer

In [14]:
#!git clone https://github.com/NVIDIA/tacotron2.git

from tacotron2.hparams import create_hparams
from tacotron2.model import Tacotron2
from tacotron2.layers import TacotronSTFT, STFT
from tacotron2.audio_processing import griffin_lim
from tacotron2.train import load_model
from tacotron2.text import text_to_sequence
from tacotron2.waveglow.mel2samp import files_to_list, MAX_WAV_VALUE
from tacotron2.denoiser import Denoiser
import numpy as np
import torch

def synthesize(text, voice, sigma=0.6, denoiser_strength=0.05, is_fp16=False):

    hparams = create_hparams()
    hparams.sampling_rate = 22050

    if voice == "papaito":
        voice_model = "nvidia_tacotron2_papaito_300"
    elif voice == "constantino":
        voice_model = "tacotron2_Constantino_600"
    elif voice == "orador":
        voice_model = "checkpoint_tacotron2_29000_es"
    
    checkpoint_path = "/home/debian/workspace/models/" + voice_model
        
    model = load_model(hparams)
    model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
    _ = model.cuda().eval().half()

    waveglow_path = '/home/debian/workspace/models/waveglow_256channels_ljs_v2.pt'
    waveglow = torch.load(waveglow_path)['model']
    _ = waveglow.cuda().eval().half()
    denoiser = Denoiser(waveglow)

    #text="¡Cágate lorito!"
    #with open(filelist_path, encoding='utf-8', mode='r') as f:
    #    text = f.read()

    sequence = np.array(text_to_sequence(text, ['english_cleaners']))[None, :]
    sequence = torch.autograd.Variable(
        torch.from_numpy(sequence)).cuda().long()

    mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
    #mel = torch.unsqueeze(mel, 0)
    mel = mel_outputs.half() if is_fp16 else mel_outputs
    audio = np.array([])
    with torch.no_grad():
        audio = waveglow.infer(mel, sigma=sigma)
        if denoiser_strength > 0:
             audio = denoiser(audio, denoiser_strength)
        audio = audio * MAX_WAV_VALUE
        audio = audio.squeeze()
        audio = audio.cpu().numpy()
        audio = audio.astype('int16')
    
    return audio, hparams.sampling_rate

## Start chat

In [18]:
import sounddevice as sd
from sty import fg, bg, ef, rs

print("¡Da-bot está listo para cascar! Escribe tus mensajes o dile 'quieto parao'")
while True:
    a = input()
    
    if a == 'quieto parao':
        break
    responses = agent.handle_text(a)
    for response in responses:
        to_synth = response["text"]
        #to_synth = "Esto es una prueba para ver si funciona"
        print(fg.blue + to_synth + fg.rs)
        response_file = open('response.txt','w') 
        response_file.write(to_synth)
        voice, sr = synthesize(to_synth, "orador")
        sd.play(voice, sr)
        response_file.close()        

¡Da-bot está listo para cascar! Escribe tus mensajes o dile 'quieto parao'
quieto parao
